In [1]:
import numpy as np
import pandas as pd
import isodata
import os
from tqdm import tqdm

In [2]:
path_home = os.getcwd()
path_data = path_home + '/data/'

In [3]:
caiso = isodata.CAISO()
print('markets:', caiso.markets)
# caiso.get_pnodes()

markets: [<Markets.REAL_TIME_15_MIN: 'REAL_TIME_15_MIN'>, <Markets.REAL_TIME_HOURLY: 'REAL_TIME_HOURLY'>, <Markets.DAY_AHEAD_HOURLY: 'DAY_AHEAD_HOURLY'>]


,Aggregate PNode ID,PNode ID
0,TH_ZP26_GEN-APND,DOUBLEC_1_B1
1,TH_SP15_GEN-APND,SONOFR2_7_B1
2,TH_SP15_GEN-APND,GARNET_7_N009
3,TH_NP15_GEN-APND,LECEFST1_7_B1
4,TH_ZP26_GEN-APND,Q885C2_7_N001
...,...,...
1589,TH_NP15_GEN-APND,TOADTWN_7_N002
1590,TH_NP15_GEN-APND,CARBERRY_2_B1
1591,TH_NP15_GEN-APND,KRMNFCP_7_N002
1592,TH_NP15_GEN-APND,MENDOTA_1_B1


In [ ]:
# write months of data to temporary location
nodes = ['TH_NP15_GEN-APND']
node_pathname = f'caiso_{nodes[0].lower()}/'
market = 'REAL_TIME_15_MIN'
sleep=5

for y in [2020, 2021, 2022]:
    for m in tqdm(range(1,12+1)):
        # define start/ end dates
        dt_start = f'{m}/1/{y}'
        if m == 12:
            dt_end = f'{1}/1/{y+1}'
        else:
            dt_end = f'{m+1}/1/{y}'

        # pull data
        caiso_month = caiso.get_historical_lmp(date=dt_start, end=dt_end, 
                                               market=market, locations=nodes, 
                                               sleep=sleep, verbose=False)
        
        # save chunk
        # If folder doesn't exist, then create it.
        path_node = f'{path_data}{node_pathname}'
        if not os.path.isdir(path_node):
            os.makedirs(path_node)
            print("created folder: ", path_node)

        caiso_month.to_csv(f'{path_node}{market.lower()}_y{y}m{m}.csv', index=False)


  8%|███▋                                        | 1/12 [00:44<08:08, 44.37s/it]

created folder:  /Users/etriesch/dev/battery-charger/data/caiso_th_np15_gen-apnd/


 42%|██████████████████▎                         | 5/12 [04:07<05:40, 48.63s/it]